In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain.retrievers import (
    BM25Retriever,
    EnsembleRetriever,
    ContextualCompressionRetriever,
)
from langchain.storage import LocalFileStore
from langchain.retrievers.multi_vector import MultiVectorRetriever, SearchType
from kiwipiepy import Kiwi
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain_teddynote import logging

from pathlib import Path
from dotenv import load_dotenv
import pickle

In [ ]:
# API 키 정보 로드
load_dotenv()

# Logging 설정
logging.langsmith("A.B.C.D.")

LangSmith 추적을 시작합니다.
[프로젝트명]
A.B.C.D.


# 앙상블 리트리버 생성
## 벡터스토어 로드 & 멀티벡터 리트리버(요약) 코드

In [ ]:
model_name = "BAAI/bge-m3"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}

hf_embeddings = HuggingFaceEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

DB_PATH = "./MultiVector_plain_final_db"
STORE_PATH = "./local_docstore"
id_key = "doc_id"

# 저장된 Chroma DB 로드
loaded_vectorstore = Chroma(
    collection_name="multivector_plain",
    embedding_function=hf_embeddings,
    persist_directory=DB_PATH,
)

# 저장된 docstore 로드
store = LocalFileStore(STORE_PATH)

# 검색기 초기화
multivector_retriever = MultiVectorRetriever(
    vectorstore=loaded_vectorstore,
    byte_store=store,
    id_key="doc_id",
)

c:\Users\SEOJUN\AppData\Local\pypoetry\Cache\virtualenvs\langchain-kr-CQxOP17p-py3.11\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
⚠️ It looks like you upgraded from a version below 0.6 and could benefit from vacuuming your database. Run chromadb utils vacuum --help for more information.


In [ ]:
def load_split_docs(file_path="./split_documents.pkl"):
    """저장된 분할 문서들을 로드"""
    if not Path(file_path).exists():
        raise FileNotFoundError(f"{file_path}를 찾을 수 없습니다.")
    with open(file_path, "rb") as f:
        loaded_docs = pickle.load(f)
    return loaded_docs

In [ ]:
docs = load_split_docs()

with open("doc_ids.pkl", "rb") as f:
    doc_ids = pickle.load(f)



multivector_retriever.docstore.mset(list(zip(doc_ids, docs)))

620개의 문서가 로드되었습니다.


In [ ]:
# 검색 유형을 MMR(Maximal Marginal Relevance)로 설정
multivector_retriever.search_type = SearchType.mmr

논리적 이미징은 디스크나 USB 메모리 등 저장 장치에서 선별된 데이터만을 대상으로 하여 하나의 포렌식 이미지 파일로 만드는 기술을 뜻한다. 오늘날 현장에서는 '선별' 방식의데이터 획득이 일반화되었기 때문에 대부분의 포렌식 이미징은 논리적 이미징 형식으로이루어진다. 특히 공공영역에서는 법 제도적으로 논리적 이미징을 디지털 포렌식 이미징의 원칙으로 삼고 이를 의무화하고 있으므로 논리적 이미징에 관한 관심은 더욱 높을 수밖에 없다. 논리적 이미징은 특히 모바일 포렌식이나 데이터베이스 포렌식, 이메일 포렌식에서 두드러지게 사용되는 기술이다. 스마트폰과 같은 모바일 디바이스에서 문자메시지 내역이나 통화내역 등을 획득해야 하는 경우 문자메시지나 통화내역과 관련된 SQLite 데이터베이스 파일만을 선별하거나 해당 데이터베이스에서 특정한 레코드만을 선별하여 자동으로보고서 형식으로 만들고 이것을 다시 논리적 이미지 파일로 만들어 획득할 수 있다. 데이터베이스 시스템에 기록된 자료에 대해서도


## KIWI + BM25 리트리버 생성 코드

In [16]:
docs = load_split_docs()

620개의 문서가 로드되었습니다.


In [ ]:
# 토큰화 함수를 생성
kiwi = Kiwi()


def kiwi_tokenize(text):
    return [token.form for token in kiwi.tokenize(text)]


# bm25 retriever
bm25_kiwi_retriever = BM25Retriever.from_documents(docs, preprocess_func=kiwi_tokenize)
bm25_kiwi_retriever.k = 3  # BM25Retriever의 검색 결과 개수

## 앙상블 리트리버 생성 코드

In [20]:
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_kiwi_retriever, multivector_retriever],
    weights=[0.2, 0.8],  # weight는 추가 조정 필요
)

# 리랭커 생성 코드

In [22]:
# Reranker
rerank_compressor = CrossEncoderReranker(
    model=HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-v2-m3"), top_n=3
)

# 최종 리트리버
retriever = ContextualCompressionRetriever(
    base_compressor=rerank_compressor, base_retriever=ensemble_retriever
)

In [23]:
retriever.invoke("이미징은 어떻게 해?")

[Document(metadata={'header1': '디지털 증거와 디지털 포렌식', 'header2': '디지털 포렌식 분석 기술', 'header3': '디지털 포렌식 기술 \\- 증거 획득', 'header4': '이미징 기술', 'source': 'data/윈도우 디지털 포렌식 완벽 활용서.md'}, page_content="이미징(Imaging)이라는 말은 포렌식 이미징(Forensic Imaging)을 줄여서 부르는 말이다. 이미징은 비트스트림 카피 (Bit\\-stream Copy) 또는 포렌식 카피 (Forensic Copy)라고 불리기도 한다.이미징은 원본인 저장 장치의 데이터를 무결하게 복제하여 하나의 논리적인 파일로 만드는 기술이다. 하나의 논리적인 파일을 생성한다는 측면에서 디스크 복제와는 차이가 있다.그리고 이미징은 디바이스 전체를 대상으로 하느냐 아니면 디바이스 일부만을 대상으로하느냐에 따라 '물리적 이미징 (Physical Imaging)'과 논리적 이미징(Logical Imaging)' 으로 구분된다.  \n##### 물리적 이미징"),
 Document(metadata={'header1': '디지털 증거 수집 및 분석 실무', 'header3': '2단계_증거물 수집', 'source': './data/감사 혁신 포렌식이 답이다.md'}, page_content='사전 준비 단계에서 획득한 정보를 바탕으로 실제 기업 등 현장을방문하여 감사대상자로부터 서면 동의를 받은 후 디지털 장치(노트북·PC·스마트 기기 등)에 대해 이미징을 수행한다. 이미징이란 전문장비를 이용하여 원본과의 동일성 및 무결성이 보장되는 형식의 파일로 복제하는 과정을 의미한다.'),
 Document(metadata={'header1': '디지털 증거와 디지털 포렌식', 'header2': '디지털 포렌식 분석 기술', 'header3': '디지털 포렌식 기술 \\- 증거 획득', 'header4': '이미징 기술', 'source': 'data/윈도우 디지